In [1]:
import numpy as np
from gensim.models import Word2Vec
from itertools import chain
from collections import Counter
import os
from tqdm import trange
import multiprocessing
import string, re

In [8]:
with open('quora_duplicate_questions.tsv', 'r') as f:
    lines = f.readlines()
    lines = [l.split('\t')[3:] for l in lines[1:]]
l = len(lines)

def clean(sent):
    sent = str(sent).lower()
    sent = ''.join('#' if i.isdigit() else i for i in sent if i not in string.punctuation)
    sent = re.sub('#+', '#', sent).split()
    return sent
data = []
for i in range(l):
    try:
        data.append([clean(lines[i][0]), clean(lines[i][1]), lines[i][2]])
    except Exception as e:
        continue
data = [[i[0], i[1], i[2]] for i in data if not (not i[0] or not i[1])]
l = len(data)

In [9]:
all_qs = np.concatenate((np.array(data)[:,0], np.array(data)[:,1]), axis=0)
vocab = Counter(chain(*all_qs))
vocab = {k:v for k, v in vocab.items() if v >= 2}
for i in range(l):
    data[i][0] = [x if x in vocab else 'unk' for x in data[i][0]]
    data[i][1] = [x if x in vocab else 'unk' for x in data[i][1]]

In [29]:
m = Word2Vec(np.concatenate([np.array(data)[:, 0], np.array(data)[:, 1]], 0), batch_words=1000, sg=1, negative=64, 
             min_count=2, iter=2, size=200, workers=multiprocessing.cpu_count())

In [4]:
# m.save('embeddings.model')
m = Word2Vec.load('embeddings.model')

In [10]:
max_length = max([len(i) for i in all_qs])
print(np.min([len(i) for i in all_qs]))
print(max_length)
batch_size = 5
dim = 200

1
237


In [5]:
i2w = dict(enumerate(list(vocab.keys()) + ['unk']))
w2i = {v:k for k, v in i2w.items()}
vocab_size = len(i2w)
for i in range(l):
    data[i][0] = [w2i[j] for j in data[i][0]] + [vocab_size] * (max_length -len(data[i][0]))
    data[i][1] = [w2i[j] for j in data[i][1]] + [vocab_size] * (max_length -len(data[i][1]))

In [6]:
def train_test_split(a, b, c, ratio = 0.8):
    mask = np.random.rand(l) < ratio
    return a[mask], b[mask], c[mask], a[~mask], b[~mask], c[~mask]

In [7]:
q1 = np.array([i[0] for i in data])
q2 = np.array([i[1] for i in data])
y = np.array([i[2] for i in data])
print(q1.shape, q2.shape, y.shape)
splitted_data = train_test_split(q1, q2, y)

(404340, 237) (404340, 237) (404340,)


In [8]:
vec = np.zeros([vocab_size + 1, dim])
for i, j in i2w.items():
    vec[i] = m[j]
vec[vocab_size] = [0] * 200

In [9]:
def batch(x, y, z, size):
    idx = np.random.choice(range(len(x)), size=size, replace=False)
    return x[idx], y[idx], z[idx]

In [10]:
rm -r data

In [11]:
if not os.path.exists('data'):
    os.mkdir('data')
for i in trange(3500):
    a, b, c = batch(q1, q2, y, 15)
    np.savez('data/'+str(i)+'.npz', ques1=a, ques2=b, label=c)

100%|██████████| 3500/3500 [05:06<00:00, 11.42it/s]


In [12]:
np.savez('data/embed.npz', embed=vec)